<a href="https://colab.research.google.com/github/jaehong31/mobilenetv2_tutorial_2022/blob/main/mobilenetv2_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/jaehong31/mobilenetv2_tutorial_2022.git

Cloning into 'mobilenetv2_tutorial_2022'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 185 (delta 19), reused 182 (delta 19), pack-reused 0
Receiving objects: 100% (185/185), 267.58 KiB | 12.74 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
%cd mobilenetv2_tutorial_2022/src

/content/mobilenetv2_tutorial_2022/src


In [ ]:
!pip install timm
!pip install -U PyYAML

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 7.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 9.3 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pwd

/content


In [ ]:
from arguments import get_args
from models import get_model
from datasets import get_dataset
import time, datetime
import torch.distributed as dist
from logger import create_logger
import numpy as np
import torch    

In [ ]:
args = get_args()
logger = create_logger(output_dir=args.log_dir, name=f"{args.tag}")   
args.train.base_lr = float(args.train.base_lr) * args.train.batch_size / 512 
args.train.warmup_lr = float(args.train.warmup_lr) * args.train.batch_size / 512 
args.train.min_lr = float(args.train.min_lr) * args.train.batch_size / 512 

creating file ../logs/in-progress_0616025656_CIFAR10-MobileNetV2
##########################################################################################################################################
args-name CIFAR10-MobileNetV2
args-log_dir ../logs/in-progress_0616025656_CIFAR10-MobileNetV2
args-ckpt_dir ../cache/CIFAR10-MobileNetV2
##########################################################################################################################################


In [ ]:
dataset = get_dataset(args)
train_loader, test_loader = dataset.get_data_loaders() 
len_train_loader = len(train_loader)
model = get_model(args, len_train_loader, logger)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../data/CIFAR10/cifar-10-python.tar.gz to ../data/CIFAR10
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


backbone info: mobilenetv2
self.n_classes 10


In [ ]:
@torch.no_grad()
def evaluate(model, test_loader, logger, loss=torch.nn.CrossEntropyLoss()):
    corrects, totals = 0, 0
    for images, labels in test_loader:
        preds = model(images.to(args.device))
        test_loss = loss(preds, labels.to(args.device))
        
        preds = preds.argmax(dim=1)
        correct = (preds == labels.to(args.device)).sum().item()               
        
        corrects += correct
        totals += preds.shape[0]
    logger.info(f'Accuracy: {(corrects/totals)*100:.2f} % ({corrects}/{totals}), Test Loss: {test_loss:.4f}')
        

In [ ]:
if hasattr(model, 'set_task'):
    logger.info(f'set task')      
    model.set_task()
  
start_time = time.time()
for epoch in range(0, args.train.num_epochs):
    start = time.time()
    model.train()

    tr_losses = 0.
    tr_p_losses = 0.
    # training phase
    for idx, (images, labels) in enumerate(train_loader):
        data_dict = model.observe(images, labels)
        tr_losses += data_dict['loss']
        tr_p_losses += data_dict['penalty']                

    if (epoch + 1) % args.eval.interval_epochs == 0:
        evaluate(model, test_loader, logger)

    epoch_time = time.time() - start
    logger.info(f"EPOCH {epoch} training takes {datetime.timedelta(seconds=int(epoch_time))}")
    logger.info('LR: {}, \
                        TR_LOSS: {}, TR_P_LOSS: {}'.format(
                            np.round(data_dict['lr'],6), 
                            np.round(tr_losses/len_train_loader, 4), 
                            np.round(tr_p_losses/len_train_loader, 4)))

if hasattr(model, 'end_task'):
    logger.info(f'end task')      
    model.set_task()

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
logger.info(f'TOTAL TRAINING TIME {total_time_str}')

[2022-06-16 03:14:56 root](<ipython-input-17-b0f1d4fcdbdd> 2): INFO set task
[2022-06-16 03:14:56 root](model.py 53): INFO optimizer_type: adam,                         warmup_lr: 1.25e-05,                         base_lr: 0.0025,                         final_lr: 0.000125
[2022-06-16 03:14:56 root](model.py 61): INFO OPTIMIZER is defined.


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[2022-06-16 03:15:53 root](<ipython-input-16-6103356d1b99> 15): INFO Accuracy: 52.26 % (5226/10000), Test Loss: 1.7029
[2022-06-16 03:15:53 root](<ipython-input-17-b0f1d4fcdbdd> 22): INFO EPOCH 0 training takes 0:00:57
[2022-06-16 03:15:53 root](<ipython-input-17-b0f1d4fcdbdd> 26): INFO LR: 0.000509,                         TR_LOSS: 1.5464, TR_P_LOSS: 0.0
[2022-06-16 03:16:50 root](<ipython-input-16-6103356d1b99> 15): INFO Accuracy: 61.44 % (6144/10000), Test Loss: 1.1900
[2022-06-16 03:16:50 root](<ipython-input-17-b0f1d4fcdbdd> 22): INFO EPOCH 1 training takes 0:00:56
[2022-06-16 03:16:50 root](<ipython-input-17-b0f1d4fcdbdd> 26): INFO LR: 0.001007,                         TR_LOSS: 1.1825, TR_P_LOSS: 0.0
[2022-06-16 03:17:46 root](<ipython-input-16-6103356d1b99> 15): INFO Accuracy: 66.81 % (6681/10000), Test Loss: 0.9118
[2022-06-16 03:17:46 root](<ipython-input-17-b0f1d4fcdbdd> 22): INFO EPOCH 2 training takes 0:00:56
[2022-06-16 03:17:46 root](<ipython-input-17-b0f1d4fcdbdd> 26): I